In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, TimeDistributed, Bidirectional, Dropout
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import shuffle
from sentence_transformers import SentenceTransformer
from keras import backend as K
nltk.download('stopwords')


d:\Capstone\.conda\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\syari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Text Cleaning Function
def clean_text(text):
    """
    This function cleans text by removing punctuation, numbers, extra whitespace, 
    and stopwords.
    """
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [3]:
# Load and Preprocess Data 
dataset = pd.read_csv('../Data/podcasts_data.csv')
dataset = shuffle(dataset)  
dataset['Podcast Name'] = dataset['Podcast Name'].apply(clean_text)
dataset['Genre'] = dataset['Genre'].apply(clean_text)


In [4]:
# Drop rows with NaN values in 'Podcast Name' column
podcast_data = dataset.dropna(subset=['Podcast Name'])

In [5]:
# Extract relevant columns
podcast_names = podcast_data['Podcast Name'].values
podcast_descriptions = podcast_data['Description'].values
podcast_publishers = podcast_data['Publisher'].values
podcast_spotify_urls = podcast_data['Spotify URL'].values
podcast_cover_image_urls = podcast_data['Cover Image URL'].values

In [6]:
# Tokenization and Vectorization
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(podcast_names)
sequences = tokenizer.texts_to_sequences(podcast_names)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
vocab_size = len(tokenizer.word_index) + 1

In [7]:
# Sentence-BERT Embedding
bert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
podcast_embeddings = bert_model.encode(podcast_names, convert_to_tensor=True)

d:\Capstone\.conda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 23, 128)           1153664   
                                                                 
 bidirectional_2 (Bidirectio  (None, 23, 128)          98816     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 23, 128)           0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 23, 128)          98816     
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 23, 128)           0         
                                                                 
 time_distributed_1 (TimeDis  (None, 23, 9013)        

In [18]:
# Prepare labels to match the output shape of the model
labels = np.expand_dims(padded_sequences, axis=-1)

# Train the model
model.fit(padded_sequences, labels, epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
106/106 [==============================] - 22s 116ms/step - loss: 2.8988 - accuracy: 0.8606 - val_loss: 1.1317 - val_accuracy: 0.8708
Epoch 2/30
106/106 [==============================] - 11s 101ms/step - loss: 1.0956 - accuracy: 0.8706 - val_loss: 1.0525 - val_accuracy: 0.8708
Epoch 3/30
106/106 [==============================] - 11s 108ms/step - loss: 1.0348 - accuracy: 0.8731 - val_loss: 1.0306 - val_accuracy: 0.8767
Epoch 4/30
106/106 [==============================] - 11s 101ms/step - loss: 1.0140 - accuracy: 0.8758 - val_loss: 1.0174 - val_accuracy: 0.8777
Epoch 5/30
106/106 [==============================] - 11s 101ms/step - loss: 0.9960 - accuracy: 0.8769 - val_loss: 0.9990 - val_accuracy: 0.8788
Epoch 6/30
106/106 [==============================] - 11s 106ms/step - loss: 0.9724 - accuracy: 0.8781 - val_loss: 0.9746 - val_accuracy: 0.8792
Epoch 7/30
106/106 [==============================] - 11s 105ms/step - loss: 0.9453 - accuracy: 0.8787 - val_loss: 0.9492 - val_ac

In [19]:
model.save('model.h5')
print('Model saved succesfully')


Model saved succesfully
